In [34]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
apps = pd.read_csv('filtered_apps.csv')
users = pd.read_csv('filtered_users.csv')
jobs = pd.read_csv('filtered_jobs.csv')

In [3]:
users.head()

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,554,1,Train,Altamonte Springs,FL,US,32701,Bachelor's,Legal Studies,2006-12-01 00:00:00,2,4.0,No,No,0
1,769,1,Test,Roselle,IL,US,60172,Bachelor's,Radio-Television,2011-05-01 00:00:00,5,5.0,Yes,No,0
2,1697,1,Train,Justice,IL,US,60458,High School,Not Applicable,2007-01-01 00:00:00,4,6.0,NaN,No,0
3,2652,1,Test,Glendale,CA,US,91204,High School,Not Applicable,1987-01-01 00:00:00,3,21.0,No,Yes,3
4,4359,1,Train,Valrico,FL,US,33594,High School,Not Applicable,2010-01-01 00:00:00,3,2.0,Yes,No,0


In [4]:
jobs.head().transpose()

,0,1,2,3,4
JobID,15,40,450,458,460
WindowID,1,1,1,1,1
Title,Administrative Assistant,Administrative Assistant,Customer Service Rep,Administrative Assistant,Data Entry - full time
Description,This Administrative Assistant position is resp...,ADMINISTRATIVE ASSISTANT Part Time The West Or...,Customer Service Rep - Provide Customer Servi...,Administrative Assistant - In Bus 26yrs Fashi...,Data Entry - full time - ...
Requirements,Please refer to the Job Description to view th...,Please refer to the Job Description to view th...,Please refer to the Job Description to view th...,Please refer to the Job Description to view th...,Qualified Candidates should have data entry sk...
City,Los Angeles,Orlando,Coral Springs,Pompano Beach,Fort Lauderdale
State,CA,FL,FL,FL,FL
Country,US,US,US,US,US
Zip5,90011.0,NaN,33065.0,33069.0,33309.0
StartDate,2012-03-09 01:12:16.81,2012-03-14 09:00:54.13,2012-03-08 01:21:44.497,2012-03-10 01:14:03.83,2012-03-11 01:18:58.9


We added the title to the job description column of the data to run the tfidf on it.

In [14]:
jobs['Title'] = jobs['Title'].fillna('')
jobs['DescCleaned'] = jobs['DescCleaned'].fillna('')
jobs['DescCleaned'] = jobs['Title'] + ' ' + jobs['DescCleaned']

We will be applying TF-IDF on the description column of the dataset.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(jobs['Description'])

In [16]:
tfidf_matrix

<6971x332029 sparse matrix of type '<class 'numpy.float64'>'
	with 1328457 stored elements in Compressed Sparse Row format>

In [17]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
cosine_sim[0]

array([1.        , 0.0652381 , 0.00246659, ..., 0.00964788, 0.01140206,
       0.04338644])

In [19]:
jobs_US_base_line = jobs.reset_index()
titles = jobs_US_base_line['Title']
indices = pd.Series(jobs.index, index=jobs['Title'])

In [20]:
def get_recommendations(query):
    # Calculate the TF-IDF vector representation of the query
    query_tfidf = tf.transform([query])

    cosine_similarities = linear_kernel(query_tfidf, tfidf_matrix).flatten()

    top_similar_indices = cosine_similarities.argsort()[::-1]

    # Get the top N closest jobs
    N = 10 
    top_N_jobs = top_similar_indices[:N]

    # Printing those top 10 job's cleaned description
    for idx in top_N_jobs:
        print(jobs.iloc[idx]['DescCleaned'])
        print()


In [22]:
get_recommendations('Customer Service Rep')

Customer Service Customer Service heavy duty truck parts company looking for customer service rep.  

Customer Service Rep Customer Service Rep spring into a fantastic career opportunity! | customer service rep | we are a firm located in a popular cincinnati neighborhood, making it difficult to leave even after the day is done! we feel that welcoming a new customer service rep through our doors will help complete our growing company! experience with customer service will be beneficial for you as our new customer service rep. if you ve always wanted a career helping others, you ll fit right in! we offer a competitive benefits package, even for your family! holidays, vacation, great salary, and more! we're looking to begin interviewing immediately, so apply now and become our newest customer service rep!

Customer Service/Sales Customer Service/Salescustomer service rep needed w/citizens & flood experience. bilingual eng/span a must. source - miami herald

Customer Service Representative

In [23]:
get_recommendations('Immediate Opening')

HR Assistant HR Assistantimmediate opening for a human resource assistant

Administrative Assistant Administrative Assistantimmediate opening for an administrative assistant. temp-hire administrative position with a large distribution center.

Medical Receptionst Medical Receptionstimmediate opening! a successful, busy, baltimore county medical practice has an immediate opening for a medical receptionist you will also be a front desk receptionist. previous experience in the medical field is a prerequisite. coding, charge posting, payment and co-payment collection. accounts receivable experience preferred. working knowledge of insurances, computerized registration, billing and collection procedures. full-time position.

Call Center Customer Service Call Center Customer Servicefast paced company in arlington is seeking an ambitious and outgoing call center representative for an immediate opening. this is a full-time temporary position lasting up to a month and is metro accessible.   

Li

In [24]:
get_recommendations('P/T HUMAN RESOURCES ASSISTANT')

Human Resources Assistant Human Resources Assistantfirm is seeking a human resources assistant with exceptional skills to join their team. human resources assistant must possess excellent verbal and written communication skills.

Human Resources Assistant / Jr Generalist Human Resources Assistant / Jr Generalist human resources assistant / jr. generalist growing trading and technology firm is looking for motivated candidates for an human resources assistant / jr generalist to join our new york based team . this person will report to the head of human resources and will provide support to the hr director and the hr team in multiple areas (benefits, payroll, new hire orientation, training, etc.). you will help to manage and update employee information in a centralized hr database. employment verifications, creating employee files, and answering general questions from employees. collegial atmosphere, great place to learn and be part of a team. jump start your career in human resources wit

In [25]:
get_recommendations('TRACTOR TRAILER DRIVER')

Office Administration Part Time - Philadelphia Office Administration Part Time - Philadelphiathe office administration position will require daily interactions with dock workers, tractor trailer drivers, delivery drivers, and customers. strong communication and organizational skills are a must. 1-2 years experience in transportation industry is preferred. this position is for second shift operating between 6 00pm and 2 00am. must have own transportation. job responsibilities receiving and responding to customers regarding international and domestic shipments preparing customs documentation communication of arrival information to customers basic data entry filing of paperwork preparing paperwork for departing loads proactively communicates with customers and resolves issues receive and enter shipments tendered through fax, e-mail, and edi in timely manner  

Administrative Management Specialist Administrative Management Specialist administrative management specialist truckway leasing an